# Extract radiomic features

In [1]:
import os
import logging
import tempfile
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor, setVerbosity
from tqdm import tqdm

# ============================================================
# Configuration
# ============================================================

INPUT_CSV = "lesions_test_t0.csv"
OUTPUT_CSV = "radiomic_features_t0.csv"
PARAMS_YAML = "params.yaml"

MIN_VOXELS = 200
DILATION_MM = 2.0
N_WORKERS = 1

setVerbosity(logging.ERROR)

# ============================================================
# Utility functions
# ============================================================

def dilate_mask_mm(mask_path, reference_img_path, mm=DILATION_MM):
    """
    Dilate a binary mask by a given radius in millimeters.
    Returns the path to a temporary dilated mask.
    """
    img = sitk.ReadImage(reference_img_path)
    mask = sitk.ReadImage(mask_path)

    spacing = np.array(img.GetSpacing())
    radius_vox = [max(1, int(round(mm / s))) for s in spacing]

    mask_dilated = sitk.BinaryDilate(mask, radius_vox, sitk.sitkBall)
    mask_dilated.CopyInformation(mask)

    tmp = tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False)
    sitk.WriteImage(mask_dilated, tmp.name)

    return tmp.name


# ============================================================
# Per-row processing
# ============================================================

def process_row(row_dict):
    """
    Extract radiomic features for a single baseline (t0) lesion.
    Returns a dictionary with features and metadata, or None if failed.
    """
    tmp_files = []

    try:
        row = pd.Series(row_dict)

        flair_path = row["t0_FLAIR_path"]
        mask_path = row["lesion_mask_t0_path"]

        if not os.path.exists(flair_path):
            return None
        if not os.path.exists(mask_path):
            return None

        extractor = featureextractor.RadiomicsFeatureExtractor(PARAMS_YAML)

        # Check mask size
        mask_img = sitk.ReadImage(mask_path)
        mask_np = sitk.GetArrayFromImage(mask_img)
        n_voxels = int((mask_np > 0).sum())

        mask_to_use = mask_path

        # Dilate small lesions
        if n_voxels < MIN_VOXELS:
            dilated_mask = dilate_mask_mm(mask_path, flair_path)
            tmp_files.append(dilated_mask)
            mask_to_use = dilated_mask

        # Extract radiomic features
        features = extractor.execute(flair_path, mask_to_use)
        features = {
            k: v for k, v in features.items()
            if not k.startswith("diagnostics_")
        }

        # Add metadata
        features.update({
            "patient_id": row["patient_id"],
            "lesion_id": row["lesion_id"],
            "mask_voxels": n_voxels,
        })

        return features

    except Exception as e:
        print(f"Error processing lesion {row_dict.get('lesion_id', '?')}: {e}")
        return None

    finally:
        for f in tmp_files:
            if os.path.exists(f):
                os.remove(f)


# ============================================================
# Main execution
# ============================================================

if __name__ == "__main__":

    df = pd.read_csv(INPUT_CSV)

    results = []

    with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
        for res in tqdm(
            executor.map(lambda r: process_row(r._asdict()),
                         df.itertuples(index=False)),
            total=len(df),
            desc="Extracting baseline radiomic features",
            ncols=100,
        ):
            if res is not None:
                results.append(res)

    radiomics_df = pd.DataFrame(results)

    meta_cols = ["patient_id", "lesion_id", "mask_voxels"]
    feature_cols = [c for c in radiomics_df.columns if c not in meta_cols]

    radiomics_df = radiomics_df[meta_cols + feature_cols]
    radiomics_df.to_csv(OUTPUT_CSV, index=False)

    print("Radiomic features extracted (baseline):")
    print(f"  → {radiomics_df.shape[0]} lesions")
    print(f"  → {radiomics_df.shape[1]} columns")

Extracting baseline radiomic features: 100%|██████████████████████████| 4/4 [00:05<00:00,  1.45s/it]

Radiomic features extracted (baseline):
  → 4 lesions
  → 105 columns


In [3]:
import pandas as pd
import numpy as np
import joblib

# ============================================================
# Paths
# ============================================================

RADIOMICS_CSV = "radiomic_features_t0.csv"
MODEL_PATH = "svm_radiomics_final.pkl"
OUT_CSV = "radiomics_with_model_output.csv"

THRESHOLD = 0.74

# ============================================================
# Load data
# ============================================================

df = pd.read_csv(RADIOMICS_CSV)

# ============================================================
# Selected radiomic features
# ============================================================

selected_feats = [
    "original_firstorder_Maximum",
    "original_shape_Sphericity",
    "original_shape_MajorAxisLength",
    "original_shape_Maximum3DDiameter",
    "original_firstorder_Mean",
    "original_shape_Maximum2DDiameterRow",
    "original_shape_Flatness",
    "original_firstorder_RootMeanSquared",
    "original_glrlm_RunVariance",
    "original_shape_Maximum2DDiameterSlice",
    "original_shape_Maximum2DDiameterColumn",
    "original_firstorder_90Percentile",
    "original_firstorder_10Percentile",
    "original_shape_MinorAxisLength",
    "original_glrlm_RunLengthNonUniformityNormalized",
    "original_shape_Elongation",
    "original_firstorder_Median",
    "original_glrlm_RunEntropy",
]

# ============================================================
# Sanity check
# ============================================================

missing = [f for f in selected_feats if f not in df.columns]
if missing:
    raise ValueError(f"Missing radiomic features: {missing}")

X = df[selected_feats].copy()

# ============================================================
# Load model
# ============================================================

model = joblib.load(MODEL_PATH)

# ============================================================
# Predict
# ============================================================

# Raw probability (class 1 = Ls-E)
if hasattr(model, "predict_proba"):
    df["svm_prob"] = model.predict_proba(X)[:, 1]
else:
    raise RuntimeError("Model does not support predict_proba; thresholding is not possible.")

# Thresholded prediction
df["svm_pred"] = (df["svm_prob"] >= THRESHOLD).astype(int)

# Optional human-readable label
df["svm_label"] = df["svm_pred"].map({0: "R-I", 1: "Ls-E"})

# ============================================================
# Save output
# ============================================================

df.to_csv(OUT_CSV, index=False)

print("Model inference completed:")
print(f"  → {df.shape[0]} lesions")
print(f"  → Threshold used: {THRESHOLD}")
print("Prediction counts:")
print(df["svm_label"].value_counts())

Model inference completed:
  → 4 lesions
  → Threshold used: 0.74
Prediction counts:
svm_label
R-I     3
Ls-E    1
Name: count, dtype: int64


## Visualization

In [21]:
# ============================================================
# 3D Visualization: All lesions in one MRI (with BET)
# ============================================================

import os
import subprocess
import tempfile
import pandas as pd
import numpy as np
import SimpleITK as sitk
import pyvista as pv

pv.global_theme.background = "white"

# ------------------------------------------------------------
# FSL BET configuration
# ------------------------------------------------------------

FSLDIR = "/Users/usuario/fsl/share/fsl"
BET_BIN = os.path.join(FSLDIR, "bin", "bet")

def run_bet(input_nifti, output_prefix, frac=0.30, overwrite=False):
    """
    Run FSL BET and return brain mask path.
    """
    mask_path = f"{output_prefix}_mask.nii.gz"

    if not overwrite and os.path.exists(mask_path):
        return mask_path

    if not os.path.exists(BET_BIN):
        raise FileNotFoundError(f"BET not found: {BET_BIN}")

    env = os.environ.copy()
    env["FSLDIR"] = FSLDIR
    env["PATH"] = os.path.join(FSLDIR, "bin") + ":" + env.get("PATH", "")
    env.setdefault("FSLOUTPUTTYPE", "NIFTI_GZ")

    cmd = [
        BET_BIN,
        input_nifti,
        output_prefix,
        "-f", str(frac),
        "-m",
    ]

    try:
        subprocess.run(
            cmd,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            env=env,
        )
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"BET failed:\n{e.stderr.decode()}")

    if not os.path.exists(mask_path):
        raise FileNotFoundError("BET did not generate brain mask")

    return mask_path


def apply_brain_mask(flair_path, brain_mask_path):
    """
    Apply brain mask to MRI and return temporary NIfTI path.
    """
    img = sitk.ReadImage(flair_path)
    mask = sitk.ReadImage(brain_mask_path, sitk.sitkUInt8)

    masked = sitk.Mask(img, mask)

    tmp = tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False)
    sitk.WriteImage(masked, tmp.name)

    return tmp.name


# ------------------------------------------------------------
# Re-attach MRI and lesion paths
# ------------------------------------------------------------

LESION_CSV = "lesions_test_t0.csv"

df_paths = pd.read_csv(LESION_CSV)[
    [
        "patient_id",
        "lesion_id",
        "t0_FLAIR_path",
        "lesion_mask_t0_path",
    ]
]

df_vis = df.merge(
    df_paths,
    on=["patient_id", "lesion_id"],
    how="left",
)

if df_vis["t0_FLAIR_path"].isna().any():
    raise RuntimeError("Some lesions are missing FLAIR paths after merge.")


# ------------------------------------------------------------
# SimpleITK → PyVista
# ------------------------------------------------------------

def sitk_to_pyvista(image):
    arr = sitk.GetArrayFromImage(image)      # (z, y, x)
    arr = np.transpose(arr, (2, 1, 0))        # (x, y, z)

    grid = pv.ImageData()
    grid.dimensions = np.array(arr.shape) + 1
    grid.spacing = image.GetSpacing()
    grid.origin = image.GetOrigin()

    grid.cell_data["values"] = arr.flatten(order="F")
    return grid


# ------------------------------------------------------------
# 3D visualization
# ------------------------------------------------------------

def plot_patient_lesions_3d(df_patient):

    flair_path = df_patient.iloc[0]["t0_FLAIR_path"]

    # --- Skull stripping
    bet_prefix = flair_path.replace(".nii.gz", "_bet")
    brain_mask = run_bet(flair_path, bet_prefix, frac=0.30)
    flair_brain_path = apply_brain_mask(flair_path, brain_mask)

    flair_img = sitk.ReadImage(flair_brain_path)
    flair_grid = sitk_to_pyvista(flair_img)

    plotter = pv.Plotter()

    opacity_tf = [0.0, 0.0, 0.007, 0.025, 0.045, 0.070, 0.095]

    plotter.add_volume(
        flair_grid,
        cmap="gray",
        opacity=opacity_tf,
        opacity_unit_distance=3.0,
    )


    # Add all lesions
    for _, row in df_patient.iterrows():

        mask_img = sitk.ReadImage(row["lesion_mask_t0_path"])
        mask_grid = sitk_to_pyvista(mask_img)

        lesion_surface = mask_grid.threshold(0.5).extract_surface()

        LESION_COLORS = {
            "Ls-E": "#0057FF",  # blue
            "R-I": "#FF8C00",   # orange
        }
        
        color = LESION_COLORS.get(row["svm_label"], "gray")

        plotter.add_mesh(
            lesion_surface,
            color=color,
            opacity=0.9,
        )

    patient_id = df_patient.iloc[0]["patient_id"]

    plotter.add_text(
        f"Patient {patient_id}",
        font_size=10,
    )

    plotter.show()


# ------------------------------------------------------------
# Run visualization
# ------------------------------------------------------------

patient_id = df_vis["patient_id"].iloc[0]   # change to select another patient
df_patient = df_vis[df_vis["patient_id"] == patient_id]

plot_patient_lesions_3d(df_patient)

Widget(value='<iframe src="http://localhost:58504/index.html?ui=P_0x146f1ff70_9&reconnect=auto" class="pyvista…